In [7]:
import sys
import os

# Add the parent directory of 'utilities' to the Python path
sys.path.append(r'C:\Users\Ai Sukmoren\Desktop\chatbot')

from dotenv import load_dotenv
from utilities.llms import llm
import os
from langchain.prompts import PromptTemplate
from langchain_community.tools import YouTubeSearchTool
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from utilities.llms import llm
import os
import ast
import re
import ast


# Load the environment variables from .env file
load_dotenv()

# Now you can access these variables using os.getenv
url=os.getenv('NE04J_CONN_URL') 
username=os.getenv('NEO4J_USERNAME') 
password=os.getenv('NEO4J_PASSWORD')
database=os.getenv('NE04J_DB')

In [9]:
def get_youtube_link(ques):
    youtube = YouTubeSearchTool()
    result = youtube.run(ques)
    # Convert the string representation of the list back into a Python list
    result_list = ast.literal_eval(result)

    # Extract the first link
    first_link = result_list[0]
    
    # Extract video ID from the URL
    video_id = re.search(r'(?<=v=)[^&#]+', first_link).group()

    # Construct the thumbnail URL
    thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

    # Use HTML to display a clickable link with a thumbnail image
    display_text = "Watch the video"  # Custom text for the link

    template = PromptTemplate.from_template(
        """
        Your task is to create a short brief about a video user is looking for in an excited tone
        Instruction:
        - Limit the word length to 50 words
    
        User prompt to find video on YouTube: {ques}
    
        """
    )
    
    response = f"""
    Here is your video link:"{first_link}" target="_blank">{display_text}</a></p>
    <img src="{thumbnail_url}" alt="Thumbnail" style="width:320px;height:180px;">
    """
    
    # Display the response as HTML
    display(HTML(response))

    return response

# Example usage
ques = "How to learn Python programming?"
get_youtube_link(ques)

'\n    Here is your video link:"https://www.youtube.com/watch?v=kqtD5dpn9C8&pp=ygUgSG93IHRvIGxlYXJuIFB5dGhvbiBwcm9ncmFtbWluZz8%3D" target="_blank">Watch the video</a></p>\n    <img src="https://img.youtube.com/vi/kqtD5dpn9C8/0.jpg" alt="Thumbnail" style="width:320px;height:180px;">\n    '

In [ ]:
graph = Neo4jGraph(url=os.getenv('NE04J_CONN_URL'), 
                    username=os.getenv('NEO4J_USERNAME'), 
                    password=os.getenv('NEO4J_PASSWORD'),
                    database=os.getenv('NE04J_DB'))

prompt = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

instruction:
- all animes is in lower cases
- all nodes include Anime and Genre

Varibales:
Schema {schema},
Question {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=prompt,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

question = "What is the top 3 highest anime rating where rating is not null"
res = cypher_chain.invoke(question)
res = res['result']
res